dowloading keras-tuner model

In [1]:
!pip install keras-tuner --upgrade

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 135 kB 5.0 MB/s 
     |████████████████████████████████| 1.6 MB 35.5 MB/s 


setting the dataset

In [2]:
from tensorflow import keras
import numpy as np

(x, y), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x[:-10000]
x_val = x[-10000:]
y_train = y[:-10000]
y_val = y[-10000:]

x_train = np.expand_dims(x_train, -1).astype("float32") / 255.0
x_val = np.expand_dims(x_val, -1).astype("float32") / 255.0
x_test = np.expand_dims(x_test, -1).astype("float32") / 255.0

num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

11490434/11490434 [==============================] - 0s 0us/step


importing the libraries

In [5]:
import keras_tuner
from keras import layers

In [6]:
def build_model(hp):
  model = keras.Sequential()
  model.add(layers.Flatten())
  model.add(layers.Dense(
      # Tune number of units
      units=hp.Int("units", min_value=32, max_value=512, step=32),
      # Tune the activation to use
      activation=hp.Choice("activation", ["relu", "tanh"]),

  ))
  # tune whether to use droput
  if hp.Boolean("dropout"):
    model.add(layers.Dropout(rate=0.25))
  model.add(layers.Dense(10, activation="softmax"))
  # Define the optimizer learning as a hyperparameter
  learning_rate = hp.Float("lr", min_value=1e-4, max_value=1e-2, sampling="log")
  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
      loss="categorical_crossentropy",
      metrics=["accuracy"],
  )
  return model

build_model(keras_tuner.HyperParameters())

Search

In [8]:
tuner = keras_tuner.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=3,
    overwrite=True,
    directory="results",
    project_name="sample",
)

Search summary

In [9]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
units (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
activation (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh'], 'ordered': False}
dropout (Boolean)
{'default': False, 'conditions': []}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


fitting the model

In [10]:
tuner.search(x_train, y_train, epochs=2, validation_data=(x_val, y_val))

Trial 3 Complete [00h 00m 49s]
val_accuracy: 0.97243332862854

Best val_accuracy So Far: 0.97243332862854
Total elapsed time: 00h 02m 07s
